# 第5单元: 使用PyTorch编写你的第一个深度强化学习算法: Reinforce. 并测试它的鲁棒性 💪

在这份笔记中, 你将从头编写你的第一个深度强化学习算法: Reinforce(也称为蒙特卡洛策略梯度).

Reinforce是一种**基于策略的方法**: 一种**尝试直接优化策略而不是使用动作价值函数**的深度强化学习算法. 更准确的说, Reinforce是策略梯度方法, 是基于策略方法的子类, 旨在**通过使用梯度提升(Gradient Ascent)估计最优策略的权重来直接优化策略.**

为了测试鲁棒性, 我们将在3个不同的简单环境进行训练:
* Cartpole-v1
* PixelcopterEnv
* PongEnv

❓如果你有任何问题, 请在discord的#study-group-unit5频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境:

* [CartPole-v1](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)
* [Pixelcopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)
* [Pong](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pong.html)

⬇️ 这是**你将在几分钟内实现的目标**的示例. ⬇️

![Sans titre.gif](./assets/Sans%20titre.gif)

## 这份笔记的目标🏆

在这份笔记学习结束后, 你将:

* 能够使用**PyTorch从头编写Reinforce算法.**
* 能够使用**简单的环境测试你的智能体的鲁棒性.**
* 能够通过精彩的回放和得分🔥**发布你训练的智能体到Hugging Face Hub.**

## 这份笔记来自深度强化学习课程
![Deep Reinforcement Learning Course.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践.**
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ **在独特的环境中训练智能体.**

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第5单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit5/README.md)

🔲 📚 通过阅读 👉 https://huggingface.co/blog/deep-rl-pg **学习策略梯度**

### 第0步: 设置GPU 💪

* 为了**更快的训练智能体, 我们将使用GPU,** 选择`修改 > 笔记本设置`
![image.png](./assets/image.png)

* `硬件加速器 > GPU`

![image.png](./assets/image1.png)

在笔记中, 我们需要生成一个回放视频. 因此在Colab(或你本地的jupyter)中, **我们需要一个虚拟屏幕能渲染环境**(记录视频帧).

下面的单元格将安装虚拟屏幕库并创建和运行虚拟屏幕. 🖥

In [ ]:
!apt install gitlfs ffmpeg
# 如果你使用IDE(例如PyCharm或VS Code)将不需要这些步骤.
!apt install python-opengl xvfb 
!pip install pyvirtualdisplay

In [ ]:
# 创建虚拟屏幕.
from pyvirtualdisplay import Display

virtual_display = Display(visible=False, size=(1400, 900))
virtual_display.start()

### 第1步: 安装依赖项 🔽
第一步是安装多个依赖项:
* `gym`: 包含Cartpole-v1, Pixelcopter和Pong环境.
* `gym-games`: 使用PyGame制作的gym环境.
* `huggingface_hub`: 🤗 是一个任何人都可以分享和探索模型和数据集的地方. 它有版本控制, 评估, 可视化和其他功能, 可以允许你简单地与他人协作.

你可以在这里看到全部可用的Reinforce模型. 👉 https://huggingface.co/models?other=reinforce

你可以在这看到全部可用的深度强化学习模型. 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning

In [ ]:
!pip install gym
!pip install git+https://github.com/ntasfi/PyGame-Learning-Environment.git
!pip install git+https://github.com/qlan3/gym-games.git
!pip install huggingface_hub
!pip install pyglet  # 如果你使用IDE, 则不需要这些步骤.

### 第2步: 导入包 📦

除了安装的库, 我们还使用:
* `imageio`: 生成回放视频.

In [ ]:
import gym

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.distributions import Categorical

* 让我们检查我们是否有GPU.
* 如果有, 你应该看到`cuda:0`或者`mps`.

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
print(device)

我们现在已经准备好实现我们的Reinforce算法 🔥

## 第一个智能体: 玩CartPole-v1 🤖️

### 第3步: 创建并理解CartPole环境
#### [环境 🎮 ](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)

![cartpole.jpg](./assets/cartpole.jpg)

### 为什么我们使用像CartPole-v1这样的简单环境?
正如[强化学习技巧](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)所解释的, 当你从头开始实现智能体时, 你需要**确保它可以正确工作, 并在深入之前找到简单环境中的bug.** 因为在简单环境中更容易找到bug.
> 尝试在玩具问题上有一些"生命迹象".

> 通过在越来越复杂的环境上运行来验证实现(你可以和RL zoo的结果进行比较). 你通常需要为该步骤运行超参数优化.
---
#### CartPole-v1环境
> 一根杆通过一个未驱动的接头连接到推车上, 该推车沿着无摩擦的轨道移动. 摆锤直立放置在推车上, 目标是通过在推车上施加左右方向的力, 使杆保持平衡.

所以, 我们从CartPole-v1开始. 目标是向左或者向右推动车辆, **使杆保持平衡.**

如果出现以下的情况, 则该局游戏结束:
* 杆的倾角超过±12˚
* 推车的位置变化超过±2.4
* 每轮游戏超过500步

杆保持平衡的每个时间步, 我们都会得到+1的奖励分数 💰.

In [ ]:
env_id = 'CartPole-v1'
# 创建环境.
env = gym.make(env_id)

# 创建评估环境.
eval_env = gym.make(env_id)

# 获取状态空间和动作空间的大小.
s_size = env.observation_space.shape
a_size = env.action_space.n

In [ ]:
print('_' * 5 + '可观察的环境' + '_' * 5, end='\n\n')
print('可观察的环境向量的形状', s_size)
print('随机采样环境', env.observation_space.sample())  # 获得一个随机的可观察环境空间.

In [ ]:
print('_' * 5 + '动作空间' + '_' * 5, end='\n\n')
print('动作的总数', a_size)
print('随机动作', env.action_space.sample())  # 获得一个随机的动作.

### 第4步: 让我们构建Reinforce架构
此实现基于两个实现:
* [PyTorch官方强化学习示例](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)
* [Udacity的Reinforce算法](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)

![image.png](./assets/image2.png)

所以我们需要:
* 两个全连接层(fc1和fc2).
* 使用ReLU作为全连接层fc1的激活函数.
* 使用Softmax输出动作的概率分布.

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 创建两个全连接层.

    def forward(self, x):
        """定义前向传播."""
        # 状态输入到fc1, 然后使用ReLU激活.
        
        # fc1输出到fc2.
        
        # 最后使用softmax激活输出.

    def act(self, state):
        """给定一个状态获得一个动作."""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state)
        m = Categorical(probs)
        action = np.argmax(m)

        return action.item(), m.log_prob(action)

#### 答案

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 创建两个全连接层.
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        """定义前向传播."""
        # 状态输入到fc1, 然后使用ReLU激活.
        x = F.relu(self.fc1(x))
        # fc1输出到fc2.
        x = self.fc2(x)
        # 最后使用softmax激活输出.
        return F.softmax(x, dim=1)

    def act(self, state):
        """给定一个状态获得一个动作."""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state)
        m = Categorical(probs)
        action = np.argmax(m)

        return action.item(), m.log_prob(action)